In [103]:
%reload_ext kedro.extras.extensions.ipython

2022-06-13 00:51:24,798 - kedro.extras.extensions.ipython - INFO - Updated path to Kedro project: C:\Users\shawn\PycharmProjects\NetworkFT\networkft
2022-06-13 00:51:24,957 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2022-06-13 00:51:25,140 - kedro.extras.extensions.ipython - WARNING - Kedro extension was registered but couldn't find a Kedro project. Make sure you run `%reload_kedro <path_to_kedro_project>`.


In [ ]:
import requests
import json
import pandas as pd
import time
from tqdm import tqdm
from pathlib import Path

In [62]:

DATA_ROOT = Path(r"C:\Users\shawn\PycharmProjects\NetworkFT\networkft\data")
folders = ["raw", "intermediate", "primary"]
DATA_PATH = {
    folder: DATA_ROOT.joinpath(f"0{i}_{folder}")
    for i, folder in enumerate(folders)
}

In [89]:
CONTRACTS = {
    "azuki": "0xED5AF388653567Af2F388E6224dC7C4b3241C544",
    "doodles": "0x8a90CAb2b38dba80c64b7734e58Ee1dB38B8992e",
    "moonbirds": "0x23581767a106ae21c074b2276D25e5C3e136a68b",
    "clonex": "0x348fc118bcc65a92dc033a951af153d14d945312",
    "mayc": "0x60E4d786628Fea6478F785A6d7e704777c86a7c6",
    "pxn": "0x160C404B2b49CBC3240055CEaEE026df1e8497A0",
    "goblintown": "0xbCe3781ae7Ca1a5e050Bd9C4c77369867eBc307e"
}

def generate_endpoint(
    topic, 
    contract_address, 
    category="tokens",
    chain_id=1,
    base_url=None
):
    """Generates the endpoint """
    base_url = base_url or "https://api.covalenthq.com/v1"
    
    endpoint = f"{chain_id}/{category}/{contract_address}/{topic}"
    url = f"{base_url}/{endpoint}/"
    return url

def get_covalent_query(url, params={}):
    """Queries the covalent API given the url endpoint and extra params"""
    params = params or {}
    params.update({"key": KEY})
    r = requests.get(url=url, params=params)
    result = r.json()
    if result["error"]:
        print(result["error_message"])
        return None
    else:
        return result["data"]

def save_json(json_file, path):
    """Saves a dict as a json in the path. Dirs will be create if it does not exist"""
    if not path.parent.is_dir():
        path.parent.mkdir(parents=True, exist_ok=False)
    with open(path, "w") as f:
        json.dump(json_file, f)
    

### Try covalent API for NFT collection

In [64]:
def parse_token_ids(result):
    """Parses covalent response into a list of ids"""
    token_ids = [item["token_id"] for item in result["items"]]
    return token_ids

In [67]:
ids[0:5]

['0', '1', '2', '3', '4']

### Get transactions for NFT contract

In [68]:
topic = "nft_transactions/94446"
url = generate_endpoint(topic, "Otherside")

In [69]:
r = requests.get(url=url, params=params)

NameError: name 'params' is not defined

In [70]:
result = r.json()

NameError: name 'r' is not defined

In [71]:
result["data"]

KeyError: 'data'

In [72]:
LOG_PARAMS = ["maker", "taker", "price"]
EVENT_PARAMS = ["block_signed_at"]

def parse_txs(response):
    """Parse response for transactions in log events"""
    txs = response["items"][0]["nft_transactions"]
    parsed_tx = [get_params_from_log(tx["log_events"]) for tx in txs]
    parsed_tx = [tx for tx in parsed_tx if tx]
    return parsed_tx


def get_params_from_log(log_events, relevant=LOG_PARAMS):
    for event in log_events:
        try:
            relevant_params = {
                param["name"]: param["value"]
                for param in event["decoded"]["params"]
                if param["name"] in LOG_PARAMS
            }
            if len(relevant_params) == len(LOG_PARAMS):
                relevant_params.update({param: event[param] for param in EVENT_PARAMS})
                # only return if all the expected value is found
                return relevant_params
        except: 
            continue
    return None

In [101]:
token_id = 9446
nft = "azuki"

url = generate_endpoint(f"nft_transactions/{token_id}", CONTRACTS[nft])
result = get_covalent_query(url)
txs = parse_txs(result)

In [24]:
txs[0:2]

[{'maker': '0x7bf9bf1e838b887b81c68fbb2a8180bbadcfa8b3',
  'taker': '0xba7741180b87d66fa4409bad86cbf88ba21867ae',
  'price': '2950000000000000000',
  'block_signed_at': '2022-06-04T13:56:57Z'},
 {'maker': '0x2c233b11d9d5cc9cc1beb1f7328c2b84b5880527',
  'taker': '0x2fa39ec3e8d1314a28e8360fe1a3dc50088153cc',
  'price': '3030000000000000000',
  'block_signed_at': '2022-06-04T00:47:58Z'}]

### Getting all transactions for a NFT

In [75]:
import itertools
import time
from tqdm import tqdm

def get_nft_txs(token_name, token_ids=None):
    """Get all transactions of all tokens for a NFT"""
    token_ids = token_ids or get_token_ids(token_name)
    params = generate_param_combinations({"token": token_name}, {"id_no": token_ids})
    txs = query_with_interval(get_token_txs, params)
    return txs
    
def query_with_interval(query, params, interval=0.2):
    """Executes API queries, following a specified time interval (in s)"""
    result = []
    for param in tqdm(params):
        try:
            time.sleep(interval)
            print(f"Querying for parameters {param}")
            result.append(query(**param))
        except Exception as e:
            print(f"Query failed for parameters {param} with exception {e}")
    return result
        
def generate_param_combinations(static_params, iterable_params):
    """Generates all possible combination of params given static and iterable ones"""
    param_combinations = []
    for iter_param in itertools.product(*iterable_params.values()):
        params = {k: v for k, v in static_params.items()}
        params.update(
            {
                arg: param for arg, param in 
                zip(iterable_params.keys(), iter_param)
            }
        )
        param_combinations.append(params)
    return param_combinations

In [275]:
get_token_txs(**{"token": "Otherside", "id_no":51})

[{'maker': '0xf7cd36d292e39845e2dd8f0987cea804f05aa012',
  'taker': '0x011c23b3aadaf3d4991f3abee262a34d18e9fdb5',
  'price': '125000000000000000000',
  'block_signed_at': '2022-05-01T14:22:42Z'}]